This code was used in "uHack Sentiments 2.0: Decode Code Words" hackathon ( https://machinehack.com/hackathon/uhack_sentiments_20_decode_code_words/leaderboard) and resulted in public score of 3.4853 - 0.75 point behind the winning submission. Still room for improvement!

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
from build_model import build_model
from preprocessing import create_tokenizer_from_hub_module, convert_single_example, \
                        convert_examples_to_features, convert_text_to_examples, initialize_vars

In [19]:
from plot_functions import plot_label

In [3]:
sess = tf.compat.v1.Session()

Let's load the data and see what our columns look like.
Note that there are multiple columns with prediction target; each column contains a binary label (0 or 1), and a text can have 1's in several columns. This means that we are dealing with multi-label classification.

In [4]:
data = pd.read_csv(r'train.csv')
data.head()

,Id,Review,Components,Delivery and Customer Support,Design and Aesthetics,Dimensions,Features,Functionality,Installation,Material,Price,Quality,Usability,Polarity
0,0,For some reason everybody complains and I'm co...,0,0,0,0,0,0,0,0,0,0,1,0
1,1,"I like everything about it, great choice of sp...",0,0,0,0,1,1,0,0,0,0,0,1
2,2,Excellent ceiling fan brace. Easy to install a...,0,0,0,0,0,0,1,0,0,1,0,1
3,3,Work great easy to use . No issues at all with...,0,0,0,0,0,1,0,0,0,0,1,1
4,4,I would recommend this product because it is p...,0,0,0,0,0,0,0,0,0,1,0,1


In [5]:
data.shape

(6136, 14)

The data contains 6136 entries - not huge, so let's try 0.1 as dev size.

In [6]:
train, dev = train_test_split(data, test_size=0.1)

In [7]:
train.shape

(5522, 14)

In [8]:
max_seq_length = 150

In [9]:
train_text = train['Review'].tolist()
train_text = np.array(train_text, dtype=object)[:, np.newaxis]

dev_text = dev['Review'].tolist()
dev_text = np.array(dev_text, dtype=object)[:, np.newaxis]

As we saw before, column 0 is ID, column 1 is text, and labels are contained in columns starting from 2

In [10]:
labels_list = data.columns[2:]
labels_list

Index(['Components', 'Delivery and Customer Support', 'Design and Aesthetics',
       'Dimensions', 'Features', 'Functionality', 'Installation', 'Material',
       'Price', 'Quality', 'Usability', 'Polarity'],
      dtype='object')

We will need the number of labels when building the model

In [11]:
num_labels = len(labels_list)
train_labels = train[labels_list].to_numpy()
dev_labels = dev[labels_list].to_numpy()

In [12]:
bert_path = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
tokenizer = create_tokenizer_from_hub_module(bert_path, sess)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Convert data to InputExample format

In [13]:
train_examples = convert_text_to_examples(train_text)
dev_examples = convert_text_to_examples(dev_text)

Convert data to BERT input format

In [14]:
(train_input_ids,
train_input_masks,
train_segment_ids) = convert_examples_to_features(tokenizer, train_examples, max_seq_length=max_seq_length)
(dev_input_ids,
dev_input_masks,
dev_segment_ids) = convert_examples_to_features(tokenizer, dev_examples, max_seq_length=max_seq_length)

Converting examples to features: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 614/614 [00:01<00:00, 472.44it/s]


In [15]:
train_input_ids.shape

(5522, 150)

In [16]:
train_labels.shape

(5522, 12)

EarlyStopping in case val_loss does not go down over 3 epochs

In [17]:
my_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                min_delta=0,
                                patience=3,
                                verbose=0,
                                mode='auto',
                                baseline=None,
                                restore_best_weights=True)

Building the model: passing max_seq_length to define the input shape, and num_labels to define the number of units in the last layer.
Compiling the model: for multi-label classifiction where each label is binary, we use binary_crossentropy loss and categorical_accuracy metric (as per https://keras.io/examples/nlp/multi_label_classification/).

In [18]:
model = build_model(max_seq_length, num_labels)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["categorical_accuracy"])
model.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 150)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 150)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 150)]        0                                            
__________________________________________________________________________________________________
bert_layer (BertLayer)          (None, 768)          110104890   input_ids[0][0]                  
                                                                 input_masks[0][0]            

In [32]:
epochs = 20
batch_size = 32

history = model.fit(
    [train_input_ids, train_input_masks, train_segment_ids],
    train_labels,
    validation_data=(
        [dev_input_ids, dev_input_masks, dev_segment_ids],
        dev_labels,
    ),
    epochs=epochs,
    batch_size=batch_size,
    callbacks=[my_callback]
)

Train on 5522 samples, validate on 614 samples
Epoch 1/20
5522/5522 [==============================] - 1397s 253ms/sample - loss: 0.2189 - categorical_accuracy: 0.2300 - my_metric: 0.0000e+00 - val_loss: 0.1940 - val_categorical_accuracy: 0.3925 - val_my_metric: 0.0000e+00
Epoch 2/20
5522/5522 [==============================] - 1393s 252ms/sample - loss: 0.1400 - categorical_accuracy: 0.3182 - my_metric: 0.0000e+00 - val_loss: 0.1807 - val_categorical_accuracy: 0.2427 - val_my_metric: 0.0000e+00
Epoch 3/20
5522/5522 [==============================] - 1398s 253ms/sample - loss: 0.1116 - categorical_accuracy: 0.3729 - my_metric: 0.0000e+00 - val_loss: 0.1732 - val_categorical_accuracy: 0.2785 - val_my_metric: 0.0000e+00
Epoch 4/20
5522/5522 [==============================] - 1400s 253ms/sample - loss: 0.0864 - categorical_accuracy: 0.3933 - my_metric: 0.0000e+00 - val_loss: 0.2007 - val_categorical_accuracy: 0.2199 - val_my_metric: 0.0000e+00
Epoch 5/20
5522/5522 [=======================

Plot model history for accuracy:

In [ ]:
plot_label(history, 'categorical_accuracy')

Plot model history for loss:

In [ ]:
plot_label(history, 'loss')

To save the model's weights:

In [33]:
model.save_weights(r'weights')

Let's predict test data labels now

In [34]:
test = pd.read_csv(r'test.csv')
test_text = test['Review'].tolist()
test_text = np.array(test_text, dtype=object)[:, np.newaxis]

In [35]:
test_examples = convert_text_to_examples(test_text)

In [36]:
(test_input_ids,
test_input_masks,
test_segment_ids) = convert_examples_to_features(tokenizer, test_examples, max_seq_length=max_seq_length)

Converting examples to features: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 2631/2631 [00:01<00:00, 2456.19it/s]


In [37]:
pred = model.predict([test_input_ids,
                        test_input_masks,
                        test_segment_ids], batch_size = batch_size)

To save predictions as a csv:

In [38]:
df = pd.DataFrame(data=pred, columns=[labels_list])
df.to_csv(r'predictions.csv', index = False)